In [1]:
import owlready2 as owl2
from urllib import request
import os

## Genereal Information


### Problem 1:

The IRI of the time ontology  <http://www.w3.org/2006/time> refers to a file in turtle syntax. This syntax is currently (2021-01-22) not supported by owlready. Thus, this ontology canont be loaded (directly) with owlready.

### Problem 2:

Another ontologies ABC which are in the supported "RDF_XML"-format might import that ontology (via `<owl:imports rdf:resource="http://www.w3.org/2006/time"/>`). Owlready detects the import, tries to resolve it and fails due to problem 1.


## Solution (Problem 1):

Show how to convert that file programmatically (i.e. without using a GUI like protégé).

- assumes https://github.com/sszuev/ont-converter to be installed (requires local build with *maven*)
- `ont-converter.jar` must be located in current dir

In [2]:
# download the original file

url = "http://www.w3.org/2006/time"
res = request.urlopen(url)
t_path = "time_turtle.txt"
with open(t_path, "wb") as t_file:
    # write plain bytes (without de- and encoding)
    t_file.write(res.read())

In [3]:
# convert from Turtle to RDF_XML

o_path = "time.owl"
cmd = f"java -jar ont-converter.jar  -i {t_path} -if TURTLE -o {o_path} -of RDF_XML"
os.system(cmd)

0

In [4]:
fpath = "time.owl"
onto = owl2.get_ontology(fpath).load()

* Owlready2 * WARNING: DataProperty http://www.w3.org/2006/time#inXSDDateTime belongs to more than one entity types: [owl.DeprecatedProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://www.w3.org/2006/time#xsdDateTime belongs to more than one entity types: [owl.DeprecatedProperty, owl.DatatypeProperty]; I'm trying to fix it...


In [5]:
list(onto.classes())

[time.DayOfWeek,
 time.Duration,
 time.TemporalDuration,
 time.DateTimeInterval,
 time.ProperInterval,
 time.TimeZone,
 time.Instant,
 time.Interval,
 time.MonthOfYear,
 time.DateTimeDescription,
 time.TemporalUnit,
 time.TemporalEntity,
 time.GeneralDateTimeDescription,
 time.DurationDescription,
 time.GeneralDurationDescription,
 time.TemporalPosition,
 time.TRS,
 time.TimePosition,
 time.Year,
 time.January]

## Solution (Problem 2):

In [6]:
world2 = owl2.World()

target = "http://ontology.eil.utoronto.ca/tove/ctime.owl"

try: 
    # while the following attempt fails, it is necessary to enable to load this ontology later
    world2.get_ontology(target).load()
except owl2.OwlReadyOntologyParsingError:
    print(f"could not import '{target}'" )

could not import 'http://ontology.eil.utoronto.ca/tove/ctime.owl'


In [7]:

# load the RDF_XML-version of the time ontology first
time_onto = world2.get_ontology(fpath).load()

# load the ontology which imports the time ontology second (this seems to require the failed attempt above)
test_onto = world2.get_ontology(target).load()

* Owlready2 * WARNING: DataProperty http://www.w3.org/2006/time#inXSDDateTime belongs to more than one entity types: [owl.DeprecatedProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://www.w3.org/2006/time#xsdDateTime belongs to more than one entity types: [owl.DeprecatedProperty, owl.DatatypeProperty]; I'm trying to fix it...


In [8]:
# manualy add the imported ontology to the respective data-structure

test_onto.imported_ontologies.append(time_onto)

In [9]:
from io import StringIO, BytesIO

In [10]:
# print the ctime-ontology
file = BytesIO()
test_onto.save(file)
print(file.getvalue().decode("utf8"))

<?xml version="1.0"?>
<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
         xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
         xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
         xmlns:owl="http://www.w3.org/2002/07/owl#"
         xml:base="http://ontology.eil.utoronto.ca/tove/ctime"
         xmlns="http://ontology.eil.utoronto.ca/tove/ctime#"
         xmlns:term="http://purl.org/dc/terms/">

<owl:Ontology rdf:about="http://ontology.eil.utoronto.ca/tove/ctime">
  <owl:imports rdf:resource="http://www.w3.org/2006/time#2016"/>
  <owl:imports rdf:resource="http://www.w3.org/2006/time"/>
  <rdfs:comment xml:lang="en">Constrained time is an extension of OWL-Time where a Cinterval is a constrained interval with hasBeginning and hasEnd ranges being a DateTime Interval that specify the min/max of the beginning and end of the interval.</rdfs:comment>
  <term:created xml:lang="en">2 February 2021 (recreation of work done a long time ago)</term:created>
  <term:cre

In [11]:
list(test_onto.classes())

[ctime.CDateTimeInterval, ctime.ctimeThing]

In [12]:
list(test_onto.properties())

[]

[ctime.ctimeThing,
 time.hasBeginning.exactly(1, time.DateTimeInterval),
 time.hasEnd.exactly(1, time.DateTimeInterval)]

### Short Version (works with freshly restarted kernel)

In [1]:
from owlready2 import *

In [2]:
try: 
    # while the following attempt fails, it is necessary to enable to load this ontology later
    ctime = default_world.get_ontology('http://ontology.eil.utoronto.ca/tove/ctime.owl').load() 
except OwlReadyOntologyParsingError:
    print(f"could not import ontology on first attempt" )


time = default_world.get_ontology('time.owl').load()
ctime = default_world.get_ontology('http://ontology.eil.utoronto.ca/tove/ctime.owl').load() 


could not import ontology on first attempt


* Owlready2 * WARNING: DataProperty http://www.w3.org/2006/time#inXSDDateTime belongs to more than one entity types: [owl.DeprecatedProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://www.w3.org/2006/time#xsdDateTime belongs to more than one entity types: [owl.DeprecatedProperty, owl.DatatypeProperty]; I'm trying to fix it...


In [3]:
# confirm that the class "CDateTimeInterval" from the ctime ontology is indeed a class-restriction
# of the original time-ontology:

ctime.CDateTimeInterval.is_a

[ctime.ctimeThing,
 time.hasBeginning.exactly(1, time.DateTimeInterval),
 time.hasEnd.exactly(1, time.DateTimeInterval)]